# Cloud Workshop Azure Databricks
## 08. Cross Validation

## Using Cross Validation

In this exercise, you will use cross-validation to optimize parameters for a regression model.

### Prepare the Data

First, import the libraries you will need and prepare the training and test data:

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

# Load the source data
# File location and type
file_location = "/FileStore/tables/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

data = data.select("DayofMonth", "DayOfWeek", "Carrier", "OriginAirportID", "DestAirportID", "DepDelay", col("ArrDelay").alias("label"))

# Split the data
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]

### Define the Pipeline
Now define a pipeline that creates a feature vector and trains a regression model

In [5]:
monthdayIndexer = StringIndexer(inputCol="DayofMonth", outputCol="DayofMonthIdx")
weekdayIndexer = StringIndexer(inputCol="DayOfWeek", outputCol="DayOfWeekIdx")
carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
originIndexer = StringIndexer(inputCol="OriginAirportID", outputCol="OriginAirportIdx")
destIndexer = StringIndexer(inputCol="DestAirportID", outputCol="DestAirportIdx")
numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normNums")

featVect = VectorAssembler(inputCols=["DayofMonthIdx", "DayOfWeekIdx", "CarrierIdx", "OriginAirportIdx", "DestAirportIdx", "normNums"], outputCol="features")

lr = LinearRegression(labelCol="label", featuresCol="features")

pipeline = Pipeline(stages=[monthdayIndexer, weekdayIndexer, carrierIndexer, originIndexer, destIndexer, numVect, minMax, featVect, lr])

### Tune Parameters
You can tune parameters to find the best model for your data. To do this you can use the  **CrossValidator** class to evaluate each combination of parameters defined in a **ParameterGrid** against multiple *folds* of the data split into training and validation datasets, in order to find the best performing parameters. Note that this can take a long time to run because every parameter combination is tried multiple times.

In [7]:
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.01]).addGrid(lr.maxIter, [10, 5]).build()
cv = CrossValidator(estimator=pipeline, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid, numFolds=2)

model = cv.fit(train)

### Test the Model
Now you're ready to apply the model to the test data.

In [9]:
prediction = model.transform(test)
predicted = prediction.select("features", "prediction", "label")

predicted.show()

+--------------------+-------------------+-----+
 features| prediction|label|
+--------------------+-------------------+-----+
[25.0,2.0,10.0,1....| 3.6578107154588935| 2|
[25.0,2.0,10.0,49...| 14.989025858213132| 7|
[25.0,2.0,10.0,49...| 93.17073512296847| 82|
[25.0,2.0,10.0,37...| 16.133966380294034| 86|
[25.0,2.0,10.0,37...| 61.00040521157471| 56|
[25.0,2.0,10.0,37...| 55.012211734153986| 55|
[25.0,2.0,10.0,13...| 4.223782615498195| 26|
[25.0,2.0,10.0,48...| -9.620098035524627| -14|
[25.0,2.0,10.0,12...| 30.294197280540708| 0|
[25.0,2.0,10.0,12...| 44.43924357085081| 57|
[25.0,2.0,10.0,16...|-0.7158738334440997| -2|
[25.0,2.0,10.0,17...| -4.699508222299464| -10|
[25.0,2.0,10.0,17...| 63.3280342836299| 29|
[25.0,2.0,10.0,67...| 4.099841111077566| -20|
[25.0,2.0,10.0,29...|-6.4772894076499625| -24|
[25.0,2.0,2.0,3.0...| 41.95011129714445| 28|
[25.0,2.0,2.0,3.0...| 73.55446930849891| 80|
[25.0,2.0,2.0,3.0...| 9.298939146652238| -4|
[25.0,2.0,2.0,3.0...|-0.8789885111084317| -10|
[25.0,2.0,2.0,3.0...| 40.987095672578185| 40|
+--------------------+-------------------+-----+
only showing top 20 rows

### Examine the Predicted and Actual Values
You can plot the predicted values against the actual values to see how accurately the model has predicted. In a perfect model, the resulting scatter plot should form a perfect diagonal line with each predicted value being identical to the actual value - in practice, some variance is to be expected.
Run the cells below to create a temporary table from the **predicted** DataFrame and then retrieve the predicted and actual label values using SQL. You can then display the results as a scatter plot, specifying **-** as the function to show the unaggregated values.

In [11]:
predicted.createOrReplaceTempView("regressionPredictions")

In [12]:
%sql
SELECT label, prediction FROM regressionPredictions

label,prediction
2,3.6578107154588935
7,14.989025858213132
82,93.17073512296847
86,16.133966380294034
56,61.00040521157471
55,55.012211734153986
26,4.223782615498195
-14,-9.620098035524627
0,30.294197280540708
57,44.43924357085081


### Retrieve the Root Mean Square Error (RMSE)
There are a number of metrics used to measure the variance between predicted and actual values. Of these, the root mean square error (RMSE) is a commonly used value that is measured in the same units as the prediced and actual values - so in this case, the RMSE indicates the average number of minutes between predicted and actual flight delay values. You can use the **RegressionEvaluator** class to retrieve the RMSE.

In [14]:
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 17.28827454111138

> Fin